In [5]:
import os

In [6]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Chicken-Disease-Classifier-With-Mlflow-and-DVC\\research'

In [7]:
os.chdir("../")

In [18]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Chicken-Disease-Classifier-With-Mlflow-and-DVC'

In [9]:
import tensorflow as tf


In [10]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [11]:
# initialize entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_image_size: list
    params_batch_size: int
    mlflow_url: str

In [14]:
%ddir

 Volume in drive C has no label.
 Volume Serial Number is CEB2-9B10

 Directory of c:\Users\hp\Documents\machine_leanrning\Deep Learning\MLOPS\Chicken-Disease-Classifier-With-Mlflow-and-DVC

01/02/2026  05:07 PM    <DIR>          .
12/30/2025  01:49 PM    <DIR>          ..
01/01/2026  05:53 PM    <DIR>          .git
12/29/2025  09:22 AM    <DIR>          .github
12/29/2025  11:02 AM    <DIR>          .vscode
01/02/2026  04:43 PM    <DIR>          artifacts
12/29/2025  09:22 AM    <DIR>          config
12/29/2025  11:29 AM    <DIR>          logs
01/02/2026  05:07 PM    <DIR>          mlruns
01/01/2026  10:45 PM    <DIR>          research
12/29/2025  11:14 AM    <DIR>          src
12/29/2025  09:22 AM    <DIR>          templates
               0 File(s)              0 bytes
              12 Dir(s)  19,192,627,200 bytes free


In [20]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories, save_json

ModuleNotFoundError: No module named 'ChickenDiseaseClassifier'

In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion"),
            mlflow_url="https://dagshub.com/am-alibee/Chicken-Disease-Classifier-With-Mlflow-and-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub

c:\Users\hp\miniconda3\envs\chest_cancer_mlops_py310\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [17]:
dagshub.init(
    repo_owner="am-alibee",
    repo_name="Chicken-Disease-Classifier-With-Mlflow-and-DVC",
    mlflow=True
)


Initialized MLflow to track repo "am-alibee/Chicken-Disease-Classifier-With-Mlflow-and-DVC"

Repository am-alibee/Chicken-Disease-Classifier-With-Mlflow-and-DVC initialized!

In [34]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config=config
    
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale=1./255,
            # validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            # directory=self.config.training_data,
            directory=os.path.join(self.config.training_data, "valid"),
            # subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def save_score(self): 
        scores={"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
 

    def log_into_mlflow(self, registered_model_name="MobileNetV2"):
        
        mlflow.set_registry_uri(self.config.mlflow_url)

        # check if backend is local store or remote
        tracking_url_type_score = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # log parameters
            mlflow.log_params(self.config.all_params)
            # log metrics
            mlflow.log_metrics({
                "loss": self.score[0], 
                "accuracy": self.score[1]
            })
            # log & register metrics if backend supports it
            if tracking_url_type_score != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name=registered_model_name
                )
                print(f"Model registered as '{registered_model_name}'")
            else:
                mlflow.keras.log_model(self.model, "model")
                print(f"Model logged to artifacts (manual registration required!)")


In [36]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-02 14:36:04,985]: INFO: common: yaml file: config\config.yaml loaded successfully:
[2026-01-02 14:36:05,008]: INFO: common: yaml file: params.yaml loaded successfully:
[2026-01-02 14:36:05,019]: INFO: common: created directory at: artifacts:
Found 72 images belonging to 4 classes.
5/5 [==============================] - 3s 167ms/step - loss: 46.2356 - accuracy: 0.2083
[2026-01-02 14:36:12,301]: INFO: common: json file saved at scores.json:


2026/01/02 14:36:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-02 14:36:43,300]: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 53). These functions will not be directly callable after loading.:
INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmppzq9e6x1\model\data\model\assets
[2026-01-02 14:36:51,471]: INFO: builder_impl: Assets written to: C:\Users\hp\AppData\Local\Temp\tmppzq9e6x1\model\data\model\assets:


Registered model 'MobileNetV2' already exists. Creating a new version of this model...
2026/01/02 14:38:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: MobileNetV2, version 2
Created version '2' of model 'MobileNetV2'.


Model registered as 'MobileNetV2'


In [28]:
print(Path)

<class 'pathlib.Path'>
